In [3]:
import sys
sys.path.insert(0, 'Resources/MagicCube/code/')
import matplotlib.pyplot as plt
import random as rd
from bitstring import BitArray
import math
import copy
import tensorflow as tf
import random

from cube import *
from utils import *

In [20]:
N = 3 #cube size

#Q-learning parameters
r = 0.15
gamma = 1 / (1 + r) #discount of the model
C = 1.
epsilon = 0.05
beta = 3./4 

In [21]:
def all_actions(N): #rotate by +90° / by -90° 
    actions = []
    c = Cube(N)
    for face_name in list(Cube.facedict.keys())[:3]: #the list is in the end ['U','D','F']
        for layer in range(c.N):
            for times in [1,-1]:
                actions.append([face_name,layer,times])
    return actions

def reward_cube(c):
    edges = computeEdges(c)
    corners = computeCorners(c)
    ncf = numCompleteFaces(c)
    nce = numCompleteEdges(c,edges)
    ncc = numCompleteCorners(c,corners)
    
    return -1 + 10*ncf + 2*nce + 3*ncc + 1000*(ncf == 6)

def state_cube(c):
    #determining the new state
    edges = computeEdges(c)
    corners = computeCorners(c)
    edges_state = []
    corners_state = []
    faces_state = []
#     for e in edges:
#         edges_state.append(e.isDone(c))
    for corner in corners:
        corners_state.append(corner.isDone(c))
    nFaces = 6
    for f in range(nFaces):
        faces_state.append(np.sum(c.stickers[f] != c.stickers[f,0,0]) == 0)
#     #conversion from binary list to int
#     e = BitArray(edges_state).uint
    c = BitArray(corners_state).uint
    f = BitArray(faces_state).uint
#     ncf = numCompleteFaces(c)
#     nce = numCompleteEdges(c,edges)
#     ncc = numCompleteCorners(c,corners)
#     return ncf,nce,ncc
    return c,f

def test_function_state_cube():
    c = Cube(3)
    print(state_cube(c))
    c.randomize(1)
    print(state_cube(c))

In [22]:
actions = all_actions(N) #rotate by +90° / by -90° 
nb_actions = len(actions)

In [23]:
c_init=Cube(3)

sess = tf.InteractiveSession()
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)) 
# with tf.device("/gpu:0"):

x = tf.placeholder(tf.float32, shape=[None,6*c_init.N**2])
act = tf.placeholder(tf.float32, shape=[nb_actions,None])
Q_ = tf.placeholder(tf.float32, shape=[None,1])

W1 = tf.Variable(tf.zeros([6*c_init.N**2,6*c_init.N**2]))
b1 = tf.Variable(tf.zeros([6*c_init.N**2]))

W2 = tf.Variable(tf.zeros([6*c_init.N**2,nb_actions]))
b2 = tf.Variable(tf.zeros([nb_actions]))

sess.run(tf.global_variables_initializer())

Q1 = tf.matmul(x,W1) + b1
Qs1 = tf.nn.relu(Q1)
Q2 = tf.nn.sigmoid(tf.matmul(Qs1,W2) + b2)
Qs = tf.matmul(Q2,act)

loss_function = tf.reduce_mean(tf.square(tf.sub(Q_,Qs)))

train_step = tf.train.GradientDescentOptimizer(1e-3).minimize(loss_function)

D = []

In [28]:
def DQN(c_init,Tmax,nb_episodes, n_moves):
    
    done = 0
    lActions = np.zeros(18)

    def eps(episode):
        return min(1,100/(1+episode))
    
    for episode in range(nb_episodes):  
        
        s = copy.deepcopy(c_init)
        s.randomize(n_moves) #we randomize n_moves times in order to have a "well mixed" cube
        cum_reward = 0

        for i in range(Tmax):
            
            #transition 
            S = np.reshape(s.stickers,(1, 54))
            if(rd.random() > eps(episode)):
                a = np.argmax(Q2.eval({x:S}))
            else:
                a = rd.randint(0,nb_actions-1)

            lActions[a] += 1
            f,l,d = actions[a]
            s.move(f,l,d)            
            R = reward_cube(s)
            cum_reward += R
            D.append([S, a, R, np.reshape(s.stickers,(1, 54))])
            
            if numCompleteFaces(s) == 6:
                done += 1
                break
                
#             print(D[0])
            Dshuf = D
            random.shuffle(Dshuf)
            batch = np.array(Dshuf[:100])
            
            tts = np.empty([0,1])
            
            for i in range(len(batch)):
                
                faces_done = np.sum([np.sum([batch[i][-1][0][f*9+j] != batch[i][-1][0][f*9] for j in range(9)]) == 0 for f in range(6)])
                
                if faces_done == 6:
                    tt = batch[i][-2]
                else:
                    tt = batch[i][-2] + gamma*max(Q2.eval({x:batch[i][-1]})[0])
             
                tts = np.append(tts,tt)
                
            tts = np.reshape(tts,(-1, 1))
            
            action = np.reshape([[i == batch[j,1] for i in range(nb_actions)] for j in range(len(batch))], (-1,len(batch)))
            train_step.run(feed_dict={Q_: tts, x: batch[:,0][0], act: action})
#             print(np.transpose(Q_.eval({Q_: tts})),Qs.eval({x:batch[:,0][0]/6, act: action})[0])
        if episode%10 == 1:
            print(episode,"\t",loss_function.eval({Q_: tts, x:batch[:,0][0], act: action}),"\t",min(Q2.eval({x:batch[:,0][0]})[0]),"\t", cum_reward, "\t", done,"\t", round(100*done/episode,2))
            print(lActions)


In [30]:
DQN(c_init=Cube(3),Tmax=1,nb_episodes=1000001,n_moves = 1)

1 	 5.76436 	 0.581594 	 27 	 1 	 100.0
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.]
11 	 6.7961 	 0.581609 	 47 	 2 	 18.18
[ 1.  0.  0.  0.  1.  0.  0.  1.  0.  0.  2.  0.  0.  1.  2.  2.  2.  0.]
21 	 5.27558 	 0.581622 	 47 	 3 	 14.29
[ 2.  0.  0.  2.  2.  0.  2.  1.  0.  1.  2.  0.  0.  1.  5.  2.  2.  0.]
31 	 9.03154 	 0.581654 	 1107 	 4 	 12.9
[ 2.  0.  1.  2.  3.  3.  2.  3.  0.  1.  2.  0.  0.  1.  6.  3.  2.  1.]
41 	 9.25523 	 0.581683 	 27 	 6 	 14.63
[ 2.  0.  1.  2.  3.  3.  3.  4.  3.  2.  2.  0.  2.  2.  6.  4.  2.  1.]
51 	 7.1967 	 0.581842 	 -1 	 7 	 13.73
[ 2.  0.  1.  2.  3.  3.  3.  4.  3.  3.  2.  0.  3.  3.  6.  6.  4.  4.]
61 	 31.2435 	 0.581939 	 3 	 8 	 13.11
[ 2.  1.  2.  2.  3.  4.  5.  6.  3.  3.  2.  0.  3.  3.  8.  6.  4.  5.]
71 	 28.3562 	 0.581958 	 15 	 9 	 12.68
[ 3.  3.  2.  4.  4.  4.  5.  7.  3.  3.  3.  0.  4.  4.  8.  6.  4.  5.]
81 	 3.89213 	 0.581974 	 27 	 9 	 11.11
[ 3.  3.  2.  4.  5.  5.  6.  8.  3.  4.  

KeyboardInterrupt: 